<a href="https://colab.research.google.com/github/Chaithanyarajbulla/Analysis-of-Superstore-Dataset/blob/main/chaithanyarajbulla_SkillChase_SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Upload the Dataset

In this step, we'll upload the **Sample - Superstore.csv** file, which contains sales data. Once the file is uploaded, we will load it into a pandas DataFrame for further processing.


In [2]:
# Step 1: Upload the dataset
from google.colab import files
import pandas as pd

# Upload the file (you will be prompted to upload 'Sample - Superstore.csv')
uploaded = files.upload()

# Try different encodings to read the CSV file
# Start with 'ISO-8859-1' or 'cp1252'
try:
    df = pd.read_csv('Sample - Superstore.csv', encoding='ISO-8859-1')
except UnicodeDecodeError:
    df = pd.read_csv('Sample - Superstore.csv', encoding='cp1252')

# Display the first few rows to verify the dataset is loaded correctly
df.head()


Saving Sample - Superstore.csv to Sample - Superstore.csv


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Step 2: Set Up SQLite Database and Create Table

In this step, we’ll set up an SQLite database in Colab and create a table based on the columns from the dataset.


In [3]:
import sqlite3

# Create a connection to an SQLite database
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

# Create the 'sales' table in SQLite
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales (
    "Row ID" INTEGER PRIMARY KEY,
    "Order ID" TEXT,
    "Order Date" TEXT,
    "Ship Date" TEXT,
    "Ship Mode" TEXT,
    "Customer ID" TEXT,
    "Customer Name" TEXT,
    "Segment" TEXT,
    "Country" TEXT,
    "City" TEXT,
    "State" TEXT,
    "Postal Code" TEXT,
    "Region" TEXT,
    "Product ID" TEXT,
    "Category" TEXT,
    "Sub-Category" TEXT,
    "Product Name" TEXT,
    Sales REAL,
    Quantity INTEGER,
    Discount REAL,
    Profit REAL
);
''')

# Insert the CSV data into the SQLite table
df.to_sql('sales', conn, if_exists='replace', index=False)

# Verify data insertion by selecting the first few rows
cursor.execute('SELECT * FROM sales LIMIT 5')
print(cursor.fetchall())


[(1, 'CA-2016-152156', '11/8/2016', '11/11/2016', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420, 'South', 'FUR-BO-10001798', 'Furniture', 'Bookcases', 'Bush Somerset Collection Bookcase', 261.96, 2, 0.0, 41.9136), (2, 'CA-2016-152156', '11/8/2016', '11/11/2016', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420, 'South', 'FUR-CH-10000454', 'Furniture', 'Chairs', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 731.94, 3, 0.0, 219.582), (3, 'CA-2016-138688', '6/12/2016', '6/16/2016', 'Second Class', 'DV-13045', 'Darrin Van Huff', 'Corporate', 'United States', 'Los Angeles', 'California', 90036, 'West', 'OFF-LA-10000240', 'Office Supplies', 'Labels', 'Self-Adhesive Address Labels for Typewriters by Universal', 14.62, 2, 0.0, 6.8714), (4, 'US-2015-108966', '10/11/2015', '10/18/2015', 'Standard Class', 'SO-20335', "Sean O'Donnell", 'Consumer', 'United States', 'Fort L

# Step 3: Run SQL Queries

Here we will execute various SQL queries to meet the project requirements. Let's start by finding the **Top Sales per Customer**.


In [4]:
query = '''
SELECT "Customer ID", "Customer Name",
       SUM(Sales) AS Total_Sales
FROM sales
GROUP BY "Customer ID", "Customer Name"
ORDER BY Total_Sales DESC;
'''

# Execute the query and display the result
result = pd.read_sql_query(query, conn)
result


,Customer ID,Customer Name,Total_Sales
0,SM-20320,Sean Miller,25043.050
1,TC-20980,Tamara Chand,19052.218
2,RB-19360,Raymond Buch,15117.339
3,TA-21385,Tom Ashbrook,14595.620
4,AB-10105,Adrian Barton,14473.571
...,...,...,...
788,RS-19870,Roy Skaria,22.328
789,MG-18205,Mitch Gastineau,16.739
790,CJ-11875,Carl Jackson,16.520
791,LD-16855,Lela Donovan,5.304


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Total_Sales'].plot(kind='hist', bins=20, title='Total_Sales')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='index', y='Total_Sales', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['Total_Sales']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('Total_Sales')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['Total_Sales'].plot(kind='line', figsize=(8, 4), title='Total_Sales')
plt.gca().spines[['top', 'right']].set_visible(False)

# **Average Discount by Product Category**


In [5]:
query = '''
SELECT Category,
       AVG(Discount) AS Average_Discount
FROM sales
GROUP BY Category;
'''

# Execute the query and display the result
result = pd.read_sql_query(query, conn)
result


,Category,Average_Discount
0,Furniture,0.173923
1,Office Supplies,0.157285
2,Technology,0.132323


from matplotlib import pyplot as plt
_df_7['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['Average_Discount'].plot(kind='hist', bins=20, title='Average_Discount')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_9.groupby('Category').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_10.plot(kind='scatter', x='index', y='Average_Discount', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_14['Average_Discount'].plot(kind='line', figsize=(8, 4), title='Average_Discount')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Category'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='index', y='Category', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['Category'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='Average_Discount', y='Category', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

# **Top 5 Cities by Total Sales**

In [6]:
query = '''
SELECT City,
       SUM(Sales) AS Total_Sales
FROM sales
GROUP BY City
ORDER BY Total_Sales DESC
LIMIT 5;
'''

# Execute the query and display the result
result = pd.read_sql_query(query, conn)
result


,City,Total_Sales
0,New York City,256368.161
1,Los Angeles,175851.341
2,Seattle,119540.742
3,San Francisco,112669.092
4,Philadelphia,109077.013


from matplotlib import pyplot as plt
_df_17['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_18['Total_Sales'].plot(kind='hist', bins=20, title='Total_Sales')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_19.groupby('City').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_20.plot(kind='scatter', x='index', y='Total_Sales', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_24['Total_Sales'].plot(kind='line', figsize=(8, 4), title='Total_Sales')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['City'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='index', y='City', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['City'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='Total_Sales', y='City', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

# **Products with Sales Greater than $5000**

In [7]:
query = '''
SELECT "Product Name",
       SUM(Sales) AS Total_Sales
FROM sales
GROUP BY "Product Name"
HAVING Total_Sales > 5000;
'''

# Execute the query and display the result
result = pd.read_sql_query(query, conn)
result


,Product Name,Total_Sales
0,"3D Systems Cube Printer, 2nd Generation, Magenta",14299.8900
1,Adjustable Depth Letter/Legal Cart,7730.1960
2,Apple iPhone 5,12996.6000
3,Ativa V4110MDD Micro-Cut Shredder,7699.8900
4,"Atlantic Metals Mobile 3-Shelf Bookcases, Cust...",7539.7122
...,...,...
102,"Tennsco Snap-Together Open Shelving Units, Sta...",6819.3120
103,Wilson Electronics DB Pro Signal Booster,8878.4000
104,Wilson SignalBoost 841262 DB PRO Amplifier Kit,5327.2600
105,Zebra GX420t Direct Thermal/Thermal Transfer P...,5787.3550


from matplotlib import pyplot as plt
_df_27['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_28['Total_Sales'].plot(kind='hist', bins=20, title='Total_Sales')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_29.plot(kind='scatter', x='index', y='Total_Sales', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['Total_Sales']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_30.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('Total_Sales')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_31.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_32['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_33['Total_Sales'].plot(kind='line', figsize=(8, 4), title='Total_Sales')
plt.gca().spines[['top', 'right']].set_visible(False)

# **Number of Orders per Shipping Mode**

In [8]:
query = '''
SELECT "Ship Mode",
       COUNT("Order ID") AS Number_of_Orders
FROM sales
GROUP BY "Ship Mode";
'''

# Execute the query and display the result
result = pd.read_sql_query(query, conn)
result


,Ship Mode,Number_of_Orders
0,First Class,1538
1,Same Day,543
2,Second Class,1945
3,Standard Class,5968


from matplotlib import pyplot as plt
_df_34['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_35['Number_of_Orders'].plot(kind='hist', bins=20, title='Number_of_Orders')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_36.groupby('Ship Mode').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_37.plot(kind='scatter', x='index', y='Number_of_Orders', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_41['Number_of_Orders'].plot(kind='line', figsize=(8, 4), title='Number_of_Orders')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_42['Ship Mode'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_42, x='index', y='Ship Mode', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_43['Ship Mode'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_43, x='Number_of_Orders', y='Ship Mode', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

# Step 4: Close the SQLite Connection

Once all queries are executed, we’ll close the SQLite database connection to ensure that all resources are properly released.


In [9]:
# Close the SQLite connection
conn.close()


# Reasons to Close the SQLite Connection

1. **Resource Management**:
   - Frees up memory and other resources used by the database connection.
   - Releases file locks, which is important if multiple processes might interact with the database.

2. **Data Integrity**:
   - Ensures that any pending changes are committed and that the database file is in a consistent state.
   - Helps maintain the integrity of the data in the database.

3. **Prevention of Errors**:
   - Avoids hitting connection limits imposed by some systems or configurations.
   - Reduces the risk of unintended changes or errors that might occur if the connection remains open longer than necessary.
